In [ ]:
import gammalib
import ctools
import cscripts 
import csv


## Create Pointing definition ASCII file in comma-separated value (CSV) format

### We set up everything we need in order to create the .csv file that will we used later on in order to create the xml file with the script csobsdef. The xml file which is created with this script can be given to the simulation tool (ctobssim). 

Choose the number of observation that you want.

In [ ]:
obs_num = 4

In [ ]:
csv_filename = 'obs_test.csv'
name = 'Crab'
ra = 83.6331
dec = 22.0145
duration = 1800        # seconds
emin = 0.05          # in TeV
emax = 20.0          # in TeV
rad = 8.0
caldb = 'prod2'
irf = 'South_0.5h'
deadc = 0.95

#for i in range(obs_num):
#    offrad = 0.5*(i/4)
#    angle = (90*i)%360
#    ra_delta = offrad*gammalib.sind(angle)
#    dec_delta = offrad*gammalib.cosd(angle)


with open(csv_filename, 'w') as csvfile:
    fieldnames = ['name', 'id','ra','dec','duration', 'emin', 'emax','']

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for id in range(1,obs_num+1):
        #routine to create some random pointings (0.5° offset step )
        # 4 pointings per each shell
        offrad = 1.0*(((id-1)/4)+1)
        angle = ((90)*(id-1))%360
        #ra_delta = offrad*gammalib.sind(angle)
        #dec_delta = offrad*gammalib.cosd(angle)
        ra_delta = 1.0
        dec_delta = 1.0
        
        id_print =str(id).rjust(4,'0')         # this create a 4-digits string filling with zeros
        writer.writerow({'name': name, 'id': id_print,'ra' : ra+ra_delta, 'dec':dec+dec_delta,'duration':duration, 'emin':emin, 'emax':emax})

In [ ]:
# this is the csv file: see http://cta.irap.omp.eu/ctools/users/reference_manual/csobsdef.html for details.
!more obs_test.csv

## Create Observation Definition XML file

All the tools can be used from Python in the same way. First you have to create an **instance** of the class, then you have to set up all the required parameters. If a python script with all the analysis chain is run from terminal (*$ python script.py*) and some informations are missing, they will be asked by the script itself.

The execute() method executes the ctobssim class in the same way as if it were executed from the command line. Alternatively, you may “run” the ctobssim tool using run(). The main difference to the execute() method is that the **run() will not write** the simulated event file to disk. Why is this useful? Well, after having typed sim.run() the ctobssim class still exists as an object in memory, including all the simulated events.

In [ ]:
# the script is called in this way because it was explicitely imported at the beginning

obsdef = cscripts.csobsdef()       #create empty instance
obsdef['inpnt'] = csv_filename     #Pointing definition ASCII file in comma-separated value (CSV) format.
obsdef['deadc'] = deadc
obsdef['caldb'] = caldb
obsdef['irf'] = irf
obsdef['rad'] = rad
obsdef['outobs'] = 'out_csobsdef.xml'
obsdef.execute()

In [ ]:
#visualization of the Observation Definition XML file.
!xmllint 'out_csobsdef.xml'

Now that we have created the **observation definition .xml** file, we can run the simulation. 

We only need to provide a **model** for the source that we want to simulate. We have to give a model for the source to be simulated: since many sources are simulated together, the xml model will have a piece for every source to simulate, including the background. An example model is $CTOOLS/share/models/crab.xml.

In [ ]:
#!xmllint $CTOOLS/share/models/crab.xml